import networkx as nx
import  numpy as np

# NODE

In this code, i choose L=even number lattice to aviod some boudary problems,and I fix all the link to node tutor parameter  and conductance,only update region to link tutor parameter,but the results show these parameter will converge. 

In [31]:
def Generator(L):
    G=nx.Graph()
    for i in range(L**2):
        x=i//L
        y=i%L
        G.add_node(i,attr='node',potensail=0,resource=np.random.rand(),position=(x,y))
    for i in G.nodes():
        x=G.node[i]['position'][0]
        y=G.node[i]['position'][1]
        neiborposition=[((x+1)%L,y),((x-1)%L,y),(x,(y-1)%L),(x,(y+1)%L)]
        for position in neiborposition:
            x1=position[0]
            y1=position[1]
            node=x1*L+y1
            G.add_edge(i,node)
    for i,j in G.edges():
        G.add_node((i,j), attr='link')
    
    for i in G.nodes():
        if G.node[i]['attr']=='node':
            x=G.node[i]['position'][0]
            y=G.node[i]['position'][1]
            regionlist=[(x,(y+1)%L),((x+1)%L,y),((x+1)%L,(y+1)%L)]
            A,B,C=[i*L+j for i,j in regionlist]
            G.add_node((i,A,B,C),attr='region')  
    for i,j in G.edges():
        G.remove_edge(i,j)
    for i in G.nodes():
        if G.node[i]['attr']=='link':
            G.add_edge(i,i[0],attr='link to node')
            G.add_edge(i,i[1],attr='link to node')
        
    for i in G.nodes():
        if G.node[i]['attr']=='region':
            a,b,c,d=i[0],i[1],i[2],i[3]
            x1,y1=sorted((a,b))
            x2,y2=sorted((a,c))
            x3,y3=sorted((b,d))
            x4,y4=sorted((c,d))
            G.add_edge(i,(x1,y1),attr='region to link')
            G.add_edge(i,(x2,y2),attr='region to link')
            G.add_edge(i,(x3,y3),attr='region to link')
            G.add_edge(i,(x4,y4),attr='region to link')
    for i,j in G.edges():
        if G.edge[i][j]['attr']=='link to node':
            G.edge[i][j]['coff']=np.random.rand()
            G.edge[i][j]['tutor']=np.random.rand()
        
    for i,j in G.edges():
        if G.edge[i][j]['attr']=='region to link':
            G.edge[i][j]['coff']={i[0]:np.random.rand(),i[1]:np.random.rand(),i:np.random.rand()}
            G.edge[i][j]['tutor']={i[0]:np.random.rand(),i[1]:np.random.rand()}
            
            
    for i in G.nodes():
        if G.node[i]['attr']=='node':
            x,y=G.node[i]['position']
            if x%2==0 and y%2==0:
                G.node[i]['kind']='s'
                G.node[i]['resource']=1
                
            elif x%2!=0 and y%2!=0:
                G.node[i]['kind']='d'
                G.node[i]['rsource']=2
            else:
                G.node[i]['kind']='r'
                G.node[i]['rsource']=-1.5
                
                
    for i,j in G.edges():
        if G.edge[i][j]['attr']=='link to node':
            m=[l for l in j if l!=i][0]
            kind1=G.node[i]['kind']
            kind2=G.node[m]['kind']
            if kind2=='d' and kind1=='r':
                G.edge[i][j]['coff']=np.sqrt(33)/6
                G.edge[i][j]['tutor']=-1
            elif kind2=='r' and kind1=='d':
                G.edge[i][j]['coff']=(15+np.sqrt(33))/32
                G.edge[i][j]['tutor']=0
            elif kind2=='s' and kind1=='r':
                G.edge[i][j]['coff']=(np.sqrt(33)+3)/12
                G.edge[i][j]['tutor']=-7/8
            elif kind2=='r' and kind1=='s':
                G.edge[i][j]['coff']=(np.sqrt(33)+3)/12
                G.edge[i][j]['tutor']=-3/8
                
                
                
    for i,j in G.edges():
        if G.edge[i][j]['attr']=='region to link':
            x,y=i[0],i[1]
            xkind,ykind=G.node[x]['kind'],G.node[y]['kind']
            if [xkind,ykind]==['s','r']or[xkind,ykind]==['r','s']:
                nodeS=[m for m in i if G.node[m]['kind']=='s'][0]
                nodeR=[m for m in i if G.node[m]['kind']=='r'][0]
                G.edge[i][j]['coff'][nodeS]=0
                G.edge[i][j]['coff'][nodeR]=0.0213
                G.edge[i][j]['coff'][i]=0
            elif [xkind,ykind]==['d','r']or[xkind,ykind]==['r','d']:
                    nodeD=[m for m in i if G.node[m]['kind']=='d'][0]
                    nodeR=[m for m in i if G.node[m]['kind']=='r'][0]
                    G.edge[i][j]['coff'][nodeD]=0.0603
                    G.edge[i][j]['coff'][nodeR]=-0.1144
                    G.edge[i][j]['coff'][i]=0.0931
                
          
    return G  

In [ ]:
def updateG(G):
    newG=G.copy()
    for a,alpha in G.edges():
        if G.edge[a][alpha]['attr']=='region to link':
            node1=a[0]
            node2=a[1]
            linklist=[m for m in G.neighbors(alpha) if m!=a]
            link16=[m for m in linklist if m[0]==a[0] or m[1]==a[0]][0]
            link25=[m for m in linklist if m[0]==a[1] or m[1]==a[1]][0]
            link56=[m for m in linklist if m!=link16 and m!=link25][0]
            gamma=[m for m in G.neighbors(link16) if G.node[m]['attr']=='region' and m!=alpha ][0]
            beta=[m for m in G.neighbors(link25) if G.node[m]['attr']=='region' and m!=alpha ][0]
            eta=[m for m in G.neighbors(link56) if G.node[m]['attr']=='region' and m!=alpha ][0]
            node6=[m for m in link16 if m!=a[0]][0]
            node5=[m for m in link25 if m!=a[1]][0]
            link68,link67=[m for m in G.neighbors(node6) if m not in [link16,link56]]
            link45,link35=[m for m in G.neighbors(node5) if m not in [link56,link25]]
            node3=[m for m in link35 if m!=node5][0]
            node4=[m for m in link45 if m!=node5][0]
            node7=[m for m in link67 if m !=node6][0]
            node8=[m for m in link68 if m!=node6][0]
            nodekind1=G.node[node1]['kind']
            nodekind2=G.node[node2]['kind']
            if nodekind1=='d' or nodekind2=='d':
                A=G.edge[node5][link35]['coff']+G.edge[node5][link45]['coff']+G.edge[link25][beta]['coff'][node5]+G.edge[link56][eta]['coff'][node5]
                B=G.edge[node5][link35]['coff']*G.edge[node5][link35]['tutor']+G.edge[node5][link45]['coff']*G.edge[node5][link45]['tutor']+G.edge[link25][beta]['coff'][node5]*G.edge[link25][beta]['tutor'][node5]+G.edge[link25][beta]['coff'][node5]*G.edge[link25][beta]['tutor'][node5]
                C=G.edge[node6][link67]['coff']+G.edge[node6][link68]['coff']+G.edge[link16][gamma]['coff'][node6]+G.edge[link56][eta]['coff'][node6]
                D=G.edge[node6][link67]['coff']*G.edge[node6][link67]['tutor']+G.edge[node6][link68]['coff']*G.edge[node6][link68]['tutor']+G.edge[link16][gamma]['coff'][node6]*G.edge[link16][gamma]['tutor'][node6]+G.edge[link56][eta]['coff'][node6]*G.edge[link56][eta]['tutor'][node6]
                a_a=1+G.edge[link56][eta]['coff'][link56]
                b=1+G.edge[link25][beta]['coff'][link25]
                c=1+G.edge[link16][gamma]['coff'][link16]
                E=(A+b+a_a)*(C+c+a_a)-a_a*a_a
                F1=G.edge[link16][gamma]['coff'][node1]*G.edge[link16][gamma]['tutor'][node1]+c/E*((A+b+a_a)*(D+G.node[node6]['resource'])+a_a*(B+G.node[node5]['resource']))
                F2=G.edge[link25][beta]['coff'][node2]*G.edge[link25][beta]['tutor'][node2]+b/E*((C+c+a_a)*(B+G.node[node5]['resource'])+a_a*(D+G.node[node6]['resource']))
                newG.edge[a][alpha]['tutor'][node1]=(F1-G.edge[node1][link16]['coff']*newG.edge[node1][link16]['tutor'])/G.edge[a][alpha]['coff'][node1]
                newG.edge[a][alpha]['tutor'][node2]=(F2-G.edge[node2][link25]['coff']*newG.edge[node2][link25]['tutor'])/G.edge[a][alpha]['coff'][node2]
            elif nodekind1=='s' and nodekind2=='r':
                A=G.edge[node5][link35]['coff']+G.edge[node5][link45]['coff']+G.edge[link25][beta]['coff'][node5]+G.edge[link56][eta]['coff'][node5]
                B=G.edge[node6][link67]['coff']+G.edge[node6][link68]['coff']+G.edge[link16][gamma]['coff'][node6]+G.edge[link56][eta]['coff'][node6]
                C=G.edge[node6][link67]['coff']*G.edge[node6][link67]['tutor']+G.edge[node6][link68]['coff']*G.edge[node6][link68]['tutor']+G.edge[link16][gamma]['coff'][node6]*G.edge[link16][gamma]['tutor'][node6]+G.edge[link56][eta]['coff'][node6]*G.edge[link56][eta]['tutor'][node6]
                a_a=1+G.edge[link56][eta]['coff'][link56]
                b=1+G.edge[link25][beta]['coff'][link25]
                c=1+G.edge[link16][gamma]['coff'][link16]
                F=G.edge[link16][gamma]['coff'][node1]*G.edge[link16][gamma]['tutor'][node1]+c*(C+G.node[node6]['resource'])/(B+a_a+c)
                newG.edge[a][alpha]['tutor'][node2]=(G.edge[link25][beta]['coff'][node2]*G.edge[link25][beta]['tutor'][node2]-G.edge[node2][link25]['coff']*newG.edge[node2][link25]['tutor'])/G.edge[a][alpha]['tutor'][node2]
            elif  nodekind1=='r' and  nodekind2=='s':
                A=G.edge[node5][link35]['coff']+G.edge[node5][link45]['coff']+G.edge[link25][beta]['coff'][node5]+G.edge[link56][eta]['coff'][node5]
                B=G.edge[node6][link67]['coff']+G.edge[node6][link68]['coff']+G.edge[link16][gamma]['coff'][node6]+G.edge[link56][eta]['coff'][node6]
                C=G.edge[node5][link35]['coff']*G.edge[node5][link35]['tutor']+G.edge[node5][link45]['coff']*G.edge[node5][link45]['tutor']+G.edge[link25][beta]['coff'][node5]*G.edge[link25][beta]['tutor'][node5]+G.edge[link25][beta]['coff'][node5]*G.edge[link25][beta]['tutor'][node5]
                a_a=1+G.edge[link56][eta]['coff'][link56]
                b=1+G.edge[link25][beta]['coff'][link25]
                c=1+G.edge[link16][gamma]['coff'][link16]
                F=G.edge[link25][beta]['coff'][node2]*G.edge[link25][beta]['tutor'][node2]+b*(C+G.node[node5]['resource'])/(A+a_a+b)
                newG.edge[a][alpha]['tutor'][node1]=(G.edge[link16][gamma]['coff'][node1]*G.edge[link16][gamma]['tutor'][node1]-G.edge[node1][link16]['coff']*newG.edge[node1][link16]['tutor'])/G.edge[a][alpha]['tutor'][node1]
    
                
                
    
    
    return newG  
    
            
        
        

## Generate graph

In [33]:
G=Generator(4) 

## update GBP algorithm

for i in range(40):
    G=updateG(G)

## show link to node tutor parameters

In [37]:
for i,j in G.edges():
    if G.edge[i][j]['attr']=='region to link':
        print(G.edge[i][j]['tutor'])

{0: 0.7930018394938947, 4: 4.3635873432238551e+140}
{0: 0.6032251733441388, 4: 6.9848416392707967e-46}
{0: 0.9325871505676168, 12: 2.4582773834695962e-08}
{0: 0.2661306474835686, 12: 1.2623719348706955e+33}
{0: 0.32785997600851524, 3: 1.7152345556874307e-09}
{0: 0.9210848242261811, 3: 310786937409654.19}
{0: 0.36630221455582335, 1: 2.8194639470682791e+134}
{0: 0.48963700918379616, 1: 1.3749886238175649e-63}
{1: -7.024529759466067e+73, 5: 3.4613046476704598e+75}
{1: -7.1697179096682548, 5: 2.9865686403039704}
{1: -7.2446409952604709, 13: 70.946060167082535}
{1: -3.6546531443568876e+73, 13: 1.8008134843255742e+75}
{1: 4.8147694634562984e-65, 2: 0.3291315959378186}
{1: 4.4899981454201184e+93, 2: 0.22570040030152472}
{2: 0.9147242920948485, 6: 1.8472397747657278e-142}
{2: 0.11330727277831854, 6: 1.1818541099428654e+37}
{2: 0.7555771377047055, 14: 5.5835565707388053e+137}
{2: 0.8393285888921863, 14: 2.8273258530049815e-17}
{2: 0.86289903848956, 3: 2.2392437079875834e+41}
{2: 0.4388388045203